In [ ]:
!pip install -q transformers accelerate decord torch torchvision facenet-pytorch
!pip install -q opencv-python-headless scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import os

os.makedirs("videos", exist_ok=True)

uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, f"videos/{filename}")

print("✅ Uploaded videos:", list(os.listdir("videos")))


Saving videoplayback (2).mp4 to videoplayback (2).mp4
✅ Uploaded videos: ['videoplayback (2).mp4']


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
from facenet_pytorch import InceptionResnetV1, MTCNN

device = "cuda" if torch.cuda.is_available() else "cpu"

# XCLIP
xclip_model = CLIPModel.from_pretrained("microsoft/xclip-base-patch32").to(device)
xclip_processor = CLIPProcessor.from_pretrained("microsoft/xclip-base-patch32")

# FaceNet + MTCNN
face_model = InceptionResnetV1(pretrained="vggface2").eval().to(device)
mtcnn = MTCNN(keep_all=True, device=device)

print("✅ Models loaded on:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

You are using a model of type xclip to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/786M [00:00<?, ?B/s]

Some weights of CLIPModel were not initialized from the model checkpoint at microsoft/xclip-base-patch32 and are newly initialized: ['vision_model.pre_layrnorm.bias', 'vision_model.pre_layrnorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

  0%|          | 0.00/107M [00:00<?, ?B/s]

✅ Models loaded on: cpu


In [ ]:
import numpy as np
from decord import VideoReader, cpu
from PIL import Image
import cv2

# Prompts
actions = ["a person speaking", "a person dancing", "a person singing", "a person painting"]
emotions = ["a person happy", "a person sad", "a person angry", "a person neutral"]

def predict_with_all_scores(frames, prompts):
    images = [Image.fromarray(f).resize((224,224)) for f in frames]
    inputs = xclip_processor(text=prompts, images=images, return_tensors="pt", padding=True).to(device)

    with torch.no_grad():
        logits_per_image = xclip_model(**inputs).logits_per_image
        probs = logits_per_image.softmax(dim=1).mean(dim=0)

    scores = [(prompts[i], float(probs[i])) for i in range(len(prompts))]
    return sorted(scores, key=lambda x: x[1], reverse=True)


In [ ]:
os.makedirs("faces_debug", exist_ok=True)

def extract_face_embeddings(frames, video_name, chunk_id):
    """Detect faces, return embeddings, and save crops for debugging."""
    embeddings = []
    meta_info = []
    frame_count = 0

    for f in frames:
        img = Image.fromarray(f)
        boxes, probs = mtcnn.detect(img)
        if boxes is not None:
            faces = mtcnn(img)
            if faces is not None:
                with torch.no_grad():
                    emb = face_model(faces.to(device)).cpu().numpy()
                for j, e in enumerate(emb):
                    embeddings.append(e)
                    meta_info.append((video_name, chunk_id))

                    # Save face crop
                    f_np = np.array(img)
                    h, w, _ = f_np.shape
                    x1,y1,x2,y2 = boxes[j].astype(int)
                    x1,y1 = max(0,x1), max(0,y1)
                    x2,y2 = min(w,x2), min(h,y2)
                    if x2 > x1 and y2 > y1:
                        crop = f_np[y1:y2, x1:x2]
                        crop_path = f"faces_debug/{video_name}_chunk{chunk_id}_frame{frame_count}_face{j}.jpg"
                        cv2.imwrite(crop_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
        frame_count += 1
    return embeddings, meta_info


In [ ]:
import pandas as pd

def analyze_video(video_path, chunks=5, frames_per_chunk=16):
    vr = VideoReader(video_path, ctx=cpu(0))
    num_frames = len(vr)
    chunk_size = num_frames // chunks

    results = []
    for i in range(chunks):
        start = i * chunk_size
        end = min((i+1) * chunk_size, num_frames)
        indices = np.linspace(start, end-1, frames_per_chunk, dtype=int)
        frames = vr.get_batch(indices).asnumpy()

        # XCLIP
        action_scores = predict_with_all_scores(frames, actions)
        emotion_scores = predict_with_all_scores(frames, emotions)

        # Face embeddings
        embeddings, meta = extract_face_embeddings(frames, os.path.basename(video_path), i+1)

        results.append({
            "Video": os.path.basename(video_path),
            "Chunk": i+1,
            "Frames": f"{start}-{end}",
            "Actions": ", ".join([f"{a}:{s:.2f}" for a,s in action_scores]),
            "Emotions": ", ".join([f"{e}:{s:.2f}" for e,s in emotion_scores]),
            "Embeddings": embeddings,
            "Embeddings_meta": meta
        })
    return results


In [ ]:
all_video_results = []

for video in os.listdir("videos"):
    video_path = os.path.join("videos", video)
    print(f"📽️ Processing {video}...")
    res = analyze_video(video_path, chunks=5, frames_per_chunk=16)
    all_video_results.extend(res)

print("✅ Finished all videos")


📽️ Processing videoplayback (2).mp4...
✅ Finished all videos


In [ ]:
from sklearn.cluster import AgglomerativeClustering

all_embeddings = []
meta_info = []

for r in all_video_results:
    for e, meta in zip(r["Embeddings"], r["Embeddings_meta"]):
        all_embeddings.append(e)
        meta_info.append(meta)

all_embeddings = np.array(all_embeddings)

# Agglomerative clustering (more stable than DBSCAN)
clustering = AgglomerativeClustering(
    n_clusters=None,
    metric="cosine",
    linkage="average",
    distance_threshold=0.7
).fit(all_embeddings)

labels = clustering.labels_

print("Clusters found:", np.unique(labels))


Clusters found: [0 1 2]


In [ ]:
import os, shutil
from PIL import Image

# Create folder for saving cropped faces
os.makedirs("person_faces", exist_ok=True)

# Remove old galleries
for p in os.listdir("person_faces"):
    path = os.path.join("person_faces", p)
    if os.path.isdir(path):
        shutil.rmtree(path)

# Assuming `all_video_results` has embeddings and labels (like before)
# and that each result has frame info, face crops, or coordinates

for idx, r in enumerate(all_video_results):
    video_name = r.get("video", f"video_{idx}")
    persons = r.get("Persons", [])
    frames = r.get("Frames", [])
    crops = r.get("Crops", [])

    for pid, frame_img in zip(persons, crops):
        folder = os.path.join("person_faces", pid)
        os.makedirs(folder, exist_ok=True)
        save_path = os.path.join(folder, f"{video_name}_{idx}.jpg")
        frame_img.save(save_path)

print("✅ All faces saved in /content/person_faces/")


✅ All faces saved in /content/person_faces/


In [ ]:
from IPython.display import HTML, display

def show_gallery(person_folder, limit=20):
    imgs = os.listdir(person_folder)[:limit]
    html = "<div style='display:flex;flex-wrap:wrap'>"
    for img in imgs:
        path = os.path.join(person_folder, img)
        html += f"<img src='{path}' width='120' height='120' style='object-fit:cover;margin:4px;border-radius:8px;border:1px solid #ccc'>"
    html += "</div>"
    display(HTML(html))

# Display first few persons' faces
for person_id in sorted(os.listdir("person_faces"))[:5]:
    print(f"🧑‍🦱 {person_id}")
    show_gallery(os.path.join("person_faces", person_id))


In [ ]:
idx = 0
for r in all_video_results:
    r["Persons"] = []
    for e in r["Embeddings"]:
        label = labels[idx]
        if label == -1:
            r["Persons"].append("Unknown")
        else:
            r["Persons"].append(f"Person_{label}")
        idx += 1


In [ ]:
final_results = []
for r in all_video_results:
    final_results.append({
        "Video": r["Video"],
        "Chunk": r["Chunk"],
        "Frames": r["Frames"],
        "Actions": r["Actions"],
        "Emotions": r["Emotions"],
        "Persons": ", ".join(set(r["Persons"])) if r["Persons"] else "No face detected"
    })

df_final = pd.DataFrame(final_results)
df_final.to_csv("video_identity_action_emotion.csv", index=False)

from google.colab import files
files.download("video_identity_action_emotion.csv")

df_final


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Video,Chunk,Frames,Actions,Emotions,Persons
0,videoplayback (2).mp4,1,0-154,"a person dancing:0.53, a person speaking:0.22,...","a person happy:0.74, a person neutral:0.18, a ...","Person_2, Person_0"
1,videoplayback (2).mp4,2,154-308,"a person dancing:0.56, a person speaking:0.20,...","a person happy:0.79, a person neutral:0.15, a ...",Person_0
2,videoplayback (2).mp4,3,308-462,"a person dancing:0.58, a person speaking:0.18,...","a person happy:0.82, a person neutral:0.13, a ...","Person_2, Person_0"
3,videoplayback (2).mp4,4,462-616,"a person dancing:0.59, a person speaking:0.19,...","a person happy:0.79, a person neutral:0.15, a ...","Person_0, Person_1"
4,videoplayback (2).mp4,5,616-770,"a person dancing:0.51, a person speaking:0.26,...","a person happy:0.68, a person neutral:0.21, a ...",Person_0


In [ ]:
from collections import defaultdict

person_summary = defaultdict(lambda: {"Videos": set(), "Actions": [], "Emotions": []})

for r in final_results:
    persons = r["Persons"].split(", ")
    for p in persons:
        if p in ["No face detected", "Unknown"]:
            continue
        person_summary[p]["Videos"].add(r["Video"])
        person_summary[p]["Actions"].append(r["Actions"])
        person_summary[p]["Emotions"].append(r["Emotions"])

summary_data = []
for person, info in person_summary.items():
    summary_data.append({
        "Person": person,
        "Videos Appeared": len(info["Videos"]),
        "Video List": ", ".join(info["Videos"]),
        "Actions": "; ".join(info["Actions"]),
        "Emotions": "; ".join(info["Emotions"])
    })

df_summary = pd.DataFrame(summary_data)
df_summary.to_csv("person_summary_report.csv", index=False)

from google.colab import files
files.download("person_summary_report.csv")

df_summary


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Person,Videos Appeared,Video List,Actions,Emotions
0,Person_2,1,videoplayback (2).mp4,"a person dancing:0.53, a person speaking:0.22,...","a person happy:0.74, a person neutral:0.18, a ..."
1,Person_0,1,videoplayback (2).mp4,"a person dancing:0.53, a person speaking:0.22,...","a person happy:0.74, a person neutral:0.18, a ..."
2,Person_1,1,videoplayback (2).mp4,"a person dancing:0.59, a person speaking:0.19,...","a person happy:0.79, a person neutral:0.15, a ..."
